### Visualization for Ground Truth* and Predicted Trajectories

List folders in the current directory

In [70]:
import json, os
print(f"Current working directory: {os.getcwd()}")
folders = {}
scan_dir = "matterport_mesh/v1/scans"
# scan_dir = "/matterport_mesh/v1/scans"
files = os.listdir(scan_dir)
print(files)
for id, name in enumerate(files):
    subfolder = os.listdir(scan_dir + "/" +name + "/matterport_mesh/")
    folders[name]=subfolder[0]

Current working directory: /home/xiezy/projects/VLN-Tutorial/visualization/topdown_map
['zsNo4HB9uLZ']


In [71]:
def idx2scan_folder(idx, trajectory_data):
    trajectory = trajectory_data[idx]
    instr_id = trajectory['instr_id']
    scan = instr_id2scan[instr_id]
    txt = instr_id2txt[instr_id]
    return [scan, folders[scan]]

def traj2conn_json(graph_path, idx, trajectory_data):
    trajectory = trajectory_data[idx]
    instr_id = trajectory['instr_id']
    scan = instr_id2scan[instr_id]
    viewpointId2idx={}
    with open(graph_path % scan) as f:
        conn_data = json.load(f)
    for i,item in enumerate(conn_data):
        viewpointId2idx[item['image_id']]=i
    return trajectory, viewpointId2idx, conn_data

def gen_conns(trajectory, viewpointId2idx, conn_data):
    trajectory = trajectory_data[idx]
    node=conn_data[viewpointId2idx[trajectory['trajectory'][0][0]]]
    node={k:v for k,v in node.items()}
    node['unobstructed'] = [False]*len(trajectory['trajectory'])
    conns=[node]
    prev_viewpoint = node['image_id']
    for n, (viewpoint, heading, elevation) in enumerate(trajectory['trajectory'][1:]):
        node=conn_data[viewpointId2idx[viewpoint]]
        node={k:v for k,v in node.items()}
        prev_viewpoint = conns[-1]['image_id']
        if viewpoint != prev_viewpoint:
            assert node['unobstructed'][viewpointId2idx[prev_viewpoint]]
            node['unobstructed'] = [False]*len(trajectory['trajectory'])
            node['unobstructed'][len(conns)-1]=True
            conns.append(node)
    return conns

In [72]:
def build_dicts(trajectory_path, instruction_path):
    with open(trajectory_path) as f:
        trajectory_data = json.load(f)
    with open(instruction_path) as f:
        instruction_data = json.load(f)

    instr_id2txt = {
        ('%s_%d' % (d['path_id'], n)): txt for d in instruction_data for n, txt in enumerate(d['instructions'])}
    instr_id2scan = {
        ('%s_%d' % (d['path_id'], n)): d['scan'] for d in instruction_data for n, txt in enumerate(d['instructions'])}
    scan2trajidx = {
        instr_id2scan[traj['instr_id']]:idx for idx, traj in enumerate(trajectory_data)}
    instr_id2trajidx = {
        traj['instr_id']:idx for idx, traj in enumerate(trajectory_data)}
    return trajectory_data, instruction_data, instr_id2txt, instr_id2scan, scan2trajidx, instr_id2trajidx

Specify the trajectory ID and modify the trajectory path

In [73]:
instr_id ="6994_0"
trajectory_path = "../../seq2seq/results/val_unseen_shortest_agent.json" #get this after run "seq2seq/eval.py"

instruction_path = "../../seq2seq/data/R2R_val_unseen.json"

graph_path = "../../Matterport3DSimulator/connectivity/%s_connectivity.json"

In [74]:
trajectory_data, instruction_data, instr_id2txt, instr_id2scan, scan2trajidx, instr_id2trajidx \
= build_dicts(trajectory_path, instruction_path)

idxs= [instr_id2trajidx[instr_id]]
scan_folders = [idx2scan_folder(idx, trajectory_data) for idx in idxs]
print(scan_folders)
instr_id = trajectory_data[idxs[0]]['instr_id']
print(instr_id)

# show instructions

instruction = instr_id2txt[instr_id]
print(instruction)
print('')
for i in ['0','1','2']:
    print(instr_id2txt[instr_id[:-1]+i])

[['zsNo4HB9uLZ', 'a641c3f4647545a2a4f5c50f5f5fbb57']]
6994_0
Exit the bedroom and enter another bedroom straight ahead, next to the table. Wait there. 

Exit the bedroom and enter another bedroom straight ahead, next to the table. Wait there. 
Walk past the foot of the bed and out the door on the other side of the room.  After passing through a small hallway, you will see two doors to bedrooms next to each other.  Enter the bedroom on the right and stop after entering the room. 
Exit bedroom.  Walk forward and enter the door on the right.  Wait inside bedroom. 


Dump files for visualization

`./jolin_mesh_names.json` and `'./%s.json'% scan` will be genereated

In [75]:
scan_folders = [idx2scan_folder(idx, trajectory_data) for idx in idxs]

with open('./jolin_mesh_names.json', 'w') as fp:
    json.dump(scan_folders, fp)

for idx, (scan, folder) in zip(idxs, scan_folders):
    with open('./%s.json'% scan, 'w') as fp:
        trajectory, viewpointId2idx, conn_data=traj2conn_json(graph_path, idx, trajectory_data)
        json.dump(gen_conns(trajectory, viewpointId2idx, conn_data), fp)

## Visualize the result through the Web Browser

### 1. Install the chrome and chrome driver

In [76]:
'''
#Check the version of chrome
/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --version

#install the chrome driver
brew install --cask chromedriver

#verify the installation
chromedriver --version
'''

'\n#Check the version of chrome\n/Applications/Google\\ Chrome.app/Contents/MacOS/Google\\ Chrome --version\n\n#install the chrome driver\nbrew install --cask chromedriver\n\n#verify the installation\nchromedriver --version\n'

### 2. Enter the folder and start http server


Run
```
cd /projects/VLN-Tutorial/visualization/topdown_map

python -m http.server 8001
```

### 3. Set the dataset in topdown_map folder
```
# First, ensure the target directory exists
mkdir -p /projects/VLN-Tutorial/visualization/topdown_map/matterport_mesh/v1/

# Then create the symbolic link
ln -s /projects/VLN-Tutorial/duet/datasets/Matterport3D/v1_unzip_scans /projects/VLN-Tutorial/visualization/topdown_map/matterport_mesh/v1/scans
```
The webpage requires the mesh files to be visualized

### 3. Start the server

In [77]:
import tempfile
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

# Create a fresh temporary directory for user data
unique_profile = tempfile.mkdtemp()
options = Options()
options.add_argument(f"--user-data-dir={unique_profile}")
service = Service(executable_path="/opt/homebrew/bin/chromedriver")

browser = webdriver.Chrome(service=service, options=options)
url = "http://127.0.0.1:8001/connectivity.html" #the url of the server in local machine
# if you run the server in remote machine, you need to change the url to the ip of the remote machine
browser.get(url)

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
imgData= browser.execute_script('return renderer.domElement.toDataURL().replace("image/png", "image/octet-stream")')
display(HTML('''<img src="%s">'''%(imgData)))

NameError: name 'browser' is not defined